<a href="https://colab.research.google.com/github/AnnaTravelblog/AnnaTravelblog/blob/main/mybot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install python-telegram-bot==13.7

In [9]:
from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext
from telegram.ext import CallbackContext
import os
import random
import requests

In [ ]:
TOKEN = '6961002291:AAEoE46aCyQJwUUqqZzVel7m10_5tvTIMYY'
OPENWEATHERMAP_API_KEY = 'f98028a3cd35a770ee7e30d447f45301'

BASE_URL = f'https://api.telegram.org/bot{TOKEN}/'
WEATHER_API_URL = 'https://api.openweathermap.org/data/2.5/weather?'

# Приветственные сообщения
GREETINGS = [
    "Привет! Я твой путешественник-бот. Готов помочь с планированием твоих следующих приключений!",
    "Здравствуй! Я готов отправиться с тобой в путешествие!",
    "Приветствую тебя, путешественник! Куда собираешься отправиться?",
    "Привет! Я здесь, чтобы помочь тебе с выбором следующего направления для путешествия."
]

DESTINATIONS = {
    "париж": "Париж - прекрасный город! Обязательно посети Эйфелеву башню и Лувр.",
    "токио": "Токио - удивительный город! Не пропусти Цукадзи и Сенсодзи.",
    "рим": "Рим - великолепный город! Посети Колизей и Ватикан.",

}

def handle_message(update: Update, context: CallbackContext) -> None:
    text = update.message.text.lower()
    destination = DESTINATIONS.get(text)
    if destination:
        weather_info = get_weather_info(text)
        send_message(update.message.chat_id, f"{destination}\n\n{weather_info}")
    elif "спасибо" in text:
        send_message(update.message.chat_id, "Пожалуйста!")
    elif "всегда пожалуйста" in text:
        send_message(update.message.chat_id, "Рад помочь!")
    else:
        send_message(update.message.chat_id, "Я не знаю такого места. Может быть, выбери что-то другое?")

def get_weather_info(city: str) -> str:
    params = {'q': city, 'appid': OPENWEATHERMAP_API_KEY, 'units': 'metric', 'lang': 'ru'}
    response = requests.get(WEATHER_API_URL, params=params)
    if response.status_code == 200:
        data = response.json()
        weather_description = data['weather'][0]['description']
        temperature = data['main']['temp']
        return f"Погода в {city.capitalize()}: {weather_description}, Температура: {temperature}°C"
    else:
        return "Информация о погоде недоступна."

def send_message(chat_id, text, reply_markup=None):
    data = {
        'chat_id': chat_id,
        'text': text,
        'reply_markup': reply_markup
    }
    response = requests.post(BASE_URL + 'sendMessage', data=data)
    print(response.json())

def start(update, context):
    update.message.reply_text(random.choice(GREETINGS) + " В какой город вы хотите поехать?")

def main():
    updater = Updater(TOKEN)
    dispatcher = updater.dispatcher

    # Обработчик команды /start
    dispatcher.add_handler(CommandHandler("start", start))

    # Добавление обработчика для обычных текстовых сообщений
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, handle_message))

    updater.start_polling()
    updater.idle()

if __name__ == '__main__':
    main()

{'ok': True, 'result': {'message_id': 183, 'from': {'id': 6961002291, 'is_bot': True, 'first_name': 'AnnaTravelblog', 'username': 'vbn222_bot'}, 'chat': {'id': 1134454737, 'first_name': 'Анна', 'last_name': 'Руд', 'username': 'AnnaRudetckaia', 'type': 'private'}, 'date': 1712599922, 'text': 'Рим - великолепный город! Посети Колизей и Ватикан.\n\nПогода в Рим: ясно, Температура: 16.44°C'}}
